In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import RGCNConv, global_mean_pool
from torch_geometric.data import Data
import pandas as pd
from torch.nn import Linear, ReLU, Sequential
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_add_pool
from sklearn.model_selection import train_test_split
import ast
from torch_geometric.utils import degree
from fractions import Fraction

In [2]:
from graph_builder import GraphBuilder  # <-- External builder

First we read the edges and coefficients of the csv files and save them in lists.

Here we read the files for 5 to 8 loops.

We aim to use 9 and 10 loop data for testing.

In [3]:
den_edges=[]
num_edges=[]
y=[]
for i in range(5, 9):
    filename = f'../Graph_Edge_Data/graph_data_{i}.csv'
    df = pd.read_csv(filename)
    den_edges += df['DEN_EDGES'].tolist()
    num_edges += df['NUM_EDGES'].tolist()
    y += df['COEFFICIENTS'].tolist()

Transform edges from strings to list of tuples.

In [4]:
den_edges = [ast.literal_eval(e) for e in den_edges]
num_edges = [ast.literal_eval(n) for n in num_edges]

Transform coefficients from strings of fractions to python Fraction. 

In [5]:
y_exact = [Fraction(s) for s in y]

Map all non zero coefficients to 1.

In [6]:
y = [0 if y0 == Fraction(0,1) else 1 for y0 in y_exact]

Build the list of Data files for each f-graph.

In [7]:
graph_list=[GraphBuilder(x,y).build() for x,y  in zip(den_edges,num_edges)]

In [8]:
class SimpleGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, num_relations=2):
        super().__init__()
        self.conv1 = RGCNConv(in_channels, hidden_channels, num_relations)
        self.conv2 = RGCNConv(hidden_channels, hidden_channels, num_relations)
        self.lin = torch.nn.Linear(hidden_channels, 1)

    def forward(self, data):
        x, edge_index, edge_type = data.x, data.edge_index, data.edge_type
        batch = torch.zeros(data.num_nodes, dtype=torch.long)  # single graph => single batch
        x = F.relu(self.conv1(x, edge_index, edge_type))
        x = F.relu(self.conv2(x, edge_index, edge_type))
        x = global_mean_pool(x, batch)  # mean over all nodes
        return self.lin(x)


Define the simple GNN model.

In [9]:
in_channels = graph_list[1].x.shape[1]
S_gnn = SimpleGNN(in_channels=in_channels, hidden_channels=16)

Evaluate the model on a single graph from the list

In [10]:
S_gnn.eval()
S_gnn(graph_list[0])

tensor([[0.0739]], grad_fn=<AddmmBackward0>)